In [ ]:
import requests
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import random
import threading
import coinconfig

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.helpers import round_step_size
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET

In [ ]:
#0:BTCBUSD, 1:BTCUSDT, 2:DOGEUSDT, 3:DOGEBUSD, 4:ETHBUSD, 5:ETHUSDT, 6:EOSUSDT, 7:LTCUSDT, 8:BNBBUSD

In [ ]:
client = Client(KEY, SECRET, testnet=False)
config=coinconfig.tradeconfig
coin=coinconfig.coinlist[4]

In [ ]:
symbol=coin["symbol"]
asset=coin["asset"]
recvWindow=config["recvWindow"]
dualSidePosition=config["dualSidePosition"]
marginType=config["marginType"]

leverage=coin["leverage"]
roundity=coin["roundity"]
profit_percent=coin["profit_percent"]
diference=coin["diference"]
stop_percent=profit_percent+diference
balance_part=coin["balance_part"]
compensation=coin["compensation"]
ingebit=coin["ingebit"]


In [ ]:
client.futures_change_leverage(symbol=symbol, leverage=leverage, recvWindow=recvWindow)

In [ ]:
# Margin type
def margin_type_to_isolated():
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    body=positions[positions['symbol']==symbol]['isolated'].astype(bool).tolist()[0]

    if body==True:
        client.futures_change_margin_type(symbol=symbol, marginType=marginType, recvWindow=recvWindow)
        print(f'Changed to {marginType}')
    else:
        print(f'Already {marginType}')

In [ ]:
margin_type_to_isolated()

In [ ]:
# Hedge mode turn on

def hedge_mode_turn_on():
    status=client.futures_get_position_mode() 
    if status['dualSidePosition']==False:
        client.futures_change_position_mode(dualSidePosition=dualSidePosition, recvWindow=recvWindow)
        client.futures_get_position_mode()
        print('Dual turned on')
    else:
        print('Already Dual')
    return(status['dualSidePosition'])

In [ ]:
hedge_mode_turn_on()

In [ ]:
def get_assetbalance(symbol):
    status = client.futures_account()
    balance = pd.DataFrame(status['assets'])
    assetbalance = float(balance.loc[balance['asset'] == asset, 'availableBalance'])
    return(assetbalance)

In [ ]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    open_positions=positions[positions['symbol']==symbol]
    position_long=float(open_positions[open_positions['positionSide']=='LONG']['entryPrice'])
    position_short=float(open_positions[open_positions['positionSide']=='SHORT']['entryPrice'])
    return(position_long,position_short)

In [ ]:
get_opened_positions(symbol)

In [ ]:
def get_quant(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    open_positions=positions[positions['symbol']==symbol]
    quant=float(open_positions[open_positions['positionSide']=='LONG']['positionAmt'])
    if quant==0:
        quant=abs(float(open_positions[open_positions['positionSide']=='SHORT']['positionAmt']))
    return(quant)

In [ ]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [ ]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [ ]:
get_symbol_price(symbol)

In [ ]:
#получить tick size

def get_tick_size(symbol):
    data = client.futures_exchange_info()
    tick_size = 0.0
    the_symbol = symbol

    found = False
    info = data['symbols']
    for s in range(len(info)):
        if info[s]['symbol'] == the_symbol:
            filters = info[s]['filters']
            for f in range(len(filters)):
                if filters[f]['filterType'] == 'PRICE_FILTER':
                    tick_size = float(filters[f]['tickSize'])
                    found = True
                    break
            break
    return(tick_size)

In [ ]:
#Trade config

# maxposition=0.016 #1 = 100% = 1 BTC,
asset_balance=get_assetbalance(symbol)
symbol_price=get_symbol_price(symbol)
maxposition=round(asset_balance*balance_part*leverage/symbol_price,3)
quantity=str(round(maxposition/2,roundity))
tick_size=get_tick_size(symbol)
symbolround=abs(str(tick_size).find('.') - len(str(tick_size))) - 1
quantity

In [ ]:
#Stop loss LONG 
def open_sl_long(symbol,quant,tick_size,stop_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[0]
        stop_price_trigger=str(round(float(entry_price)*(1-stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }

        open_position_sl_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_sl_opened=float(open_position_sl_long_response[0]['price'])
        print('LONG sl order opened price:'+str(long_sl_opened))
        return(long_sl_opened)
    
    except :
        print('\n\nSL long not open...')
        

In [ ]:
#Take Profit LONG 

def open_tp_long(symbol,quant,tick_size,profit_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[0]
        take_price_trigger=str(round(float(entry_price)*(1+profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }
        open_position_tp_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_tp_opened=float(open_position_tp_long_response[0]['price'])
        print('LONG tp order opened price:'+str(long_tp_opened))
        return(long_tp_opened)
        
    except :
        print('\n\nTP long not open...')
    return(profit_percent,stop_percent)

In [ ]:
#Открыть sl&tp LONG если есть позиция

def long_tp_sl_opening(symbol,quantity,profit_percent,stop_percent):
    position=get_opened_positions(symbol)
    if position[0]>0:
        orders=client.futures_get_open_orders(symbol=symbol)
        orders_df=pd.DataFrame(orders)
        if orders_df.empty:
            x = threading.Thread(target=open_tp_long, args=(symbol,quantity,tick_size,profit_percent))
            x.start()
            y = threading.Thread(target=open_sl_long, args=(symbol,quantity,tick_size,stop_percent))
            y.start()
        else:
            order_long=orders_df[orders_df['positionSide']=='LONG']
            long_tp=order_long[order_long['origType']=='TAKE_PROFIT']['price']
            long_sl=order_long[order_long['origType']=='STOP']['price']
            if (long_tp+long_sl).empty:
                x = threading.Thread(target=open_tp_long, args=(symbol,quantity,tick_size,profit_percent))
                x.start()
                y = threading.Thread(target=open_sl_long, args=(symbol,quantity,tick_size,stop_percent))
                y.start()
            else:
                print('LONG SL TP iteration continious..')

In [ ]:
#Открыть позицию LONG 

def open_position_long(symbol,quantity,tick_size,profit_percent,stop_percent):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1+0.002),symbolround))
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "BUY",
                "positionSide": "LONG",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    long_opened=float(open_position_long_response[0]['price'])
    print('LONG order opened price:'+str(long_opened))
    if long_opened>0:
        long_tp_sl=long_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)
    return(long_opened)

In [ ]:
#Stop loss SHORT 

def open_sl_short(symbol,quant,tick_size,stop_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[1]
        stop_price_trigger=str(round(float(entry_price)*(1+stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }
        
        open_position_sl_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_sl_opened=float(open_position_sl_short_response[0]['price'])
        print('SHORT sl order opened price:'+str(short_sl_opened))
        return(short_sl_opened)
    
    except :
        print('\n\nSL short not open...')


In [ ]:
#Take Profit SHORT

def open_tp_short(symbol,quant,tick_size,profit_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[1]
        take_price_trigger=str(round(float(entry_price)*(1-profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }

        open_position_tp_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_tp_opened=float(open_position_tp_short_response[0]['price'])
        print('SHORT tp order opened price:'+str(short_tp_opened))
        return(short_tp_opened)
    
    except :
        print('\n\nTp short not open...')


In [ ]:
#Открыть sl&tp SHORT если есть позиция
       
def short_tp_sl_opening(symbol,quantity,profit_percent,stop_percent):
    position=get_opened_positions(symbol)
    if position[1]>0:
        orders=client.futures_get_open_orders(symbol=symbol)
        orders_df=pd.DataFrame(orders)
        if orders_df.empty:
            x = threading.Thread(target=open_tp_short, args=(symbol,quantity,tick_size,profit_percent))
            x.start()
            y = threading.Thread(target=open_sl_short, args=(symbol,quantity,tick_size,stop_percent))
            y.start()
        else:
            order_short=orders_df[orders_df['positionSide']=='SHORT']
            short_tp=order_short[order_short['origType']=='TAKE_PROFIT']['price']
            short_sl=order_short[order_short['origType']=='STOP']['price']
            if (short_tp+short_sl).empty:
                x = threading.Thread(target=open_tp_short, args=(symbol,quantity,tick_size,profit_percent))
                x.start()
                y = threading.Thread(target=open_sl_short, args=(symbol,quantity,tick_size,stop_percent))
                y.start()
            else:
                print('SHORT SL TP iteration continious..')

In [ ]:
#Открыть позицию SHORT

def open_position_short(symbol,quantity,tick_size,profit_percent,stop_percent):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1-0.002),symbolround))
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "SELL",
                "positionSide": "SHORT",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    short_opened=float(open_position_short_response[0]['price'])
    print('SHORT order opened price:'+str(short_opened))
    if short_opened>0:
        short_tp_sl=short_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)
    return(short_opened)

In [ ]:
def get_profit(symbol):
    trades = client.futures_account_trades(symbol=symbol)
    dataset=pd.DataFrame(trades).iloc[-10:]
    bnbcom=dataset[dataset['commissionAsset']=='BNB']['commission'].astype(float).sum()
    busdcom=dataset[dataset['commissionAsset']==asset]['commission'].astype(float).sum()
    com=(bnbcom*270+busdcom)*-1
    real=dataset['realizedPnl'].astype(float).sum()
    profit=float(com+real)
    dataseto=pd.DataFrame(trades)['realizedPnl'].astype(float)[-50:]
    dataseto=pd.DataFrame(dataseto)
    datasetneg=len(dataseto[dataseto['realizedPnl']<0].index)
    datasetpos=len(dataseto[dataseto['realizedPnl']>0].index)
    print(profit,'+',datasetpos,'-',datasetneg)

In [ ]:
get_profit(symbol)

In [ ]:
def get_direction(symbol):
    trades = client.futures_account_trades(symbol=symbol)[-1:]
    dataset = pd.DataFrame(trades)
    resdata = float(dataset['realizedPnl'])
    direction=1
    if resdata < 0:
        direction=0
    return(direction)

In [ ]:
get_direction(symbol)

In [ ]:
def changer(symbol):
    direction=get_direction(symbol)
    f = open("directdoc.txt", "r")
    direc=f.read()
    # stop=stop_percent/ingebit
    # profit=profit_percent/ingebit
    # quant=str(round(float(quantity)/ingebit,roundity))
    if direction==0: #не меняем
        if direc=='0':
            open_position_long(symbol,quantity,tick_size,profit_percent,profit_percent)
            f = open("directdoc.txt", "w")
            f.write("1")
            f.close()
            print('Direction switched from SHORT to LONG')
        else:
            open_position_short(symbol,quantity,tick_size,profit_percent,profit_percent)
            print('Direction switched from LONG to SHORT')
            f = open("directdoc.txt", "w")
            f.write("0")
            f.close()
    else: #реверс
        if direc=='0':
            target=open_position_short(symbol,quantity,tick_size,profit_percent,profit_percent)
            print('Direction SHORT again')
            f = open("directdoc.txt", "w")
            f.write("0")
            f.close()
        if direc=='1':
            target=open_position_long(symbol,quantity,tick_size,profit_percent,profit_percent)
            print('Direction LONG again')
            f = open("directdoc.txt", "w")
            f.write("1")
            f.close()

In [ ]:
def check_positions_new(symbol):
    positions=get_opened_positions(symbol)
    if positions[0]+positions[1]==0:
        orders=client.futures_get_open_orders(symbol=symbol)
        orders_df=pd.DataFrame(orders)
        if orders_df.empty:
            changer(symbol)
    else:
        quant=str(get_quant(symbol))
        if positions[0]>0 and positions[1]==0:
            sprice=get_symbol_price(symbol)
            if sprice<positions[0]*(1-compensation):
                
                stopy=(positions[0]-sprice)/positions[0]
                profy=profit_percent-stopy
                open_position_short(symbol,quant,tick_size,(profit_percent-compensation),compensation)
                print('SHORT compensation opened')
            else:
                print('SHORT compensation not in condition')
        if positions[1]>0 and positions[0]==0:
            sprice=get_symbol_price(symbol)
            if sprice>positions[1]*(1+compensation):
                                
                stopy=(sprice-positions[1])/sprice
                profy=profit_percent-stopy
                open_position_long(symbol,quant,tick_size,(profit_percent-compensation),compensation)
                print('LONG compensation opened')
            else:
                print('LONG compensation not in condition')

In [ ]:
def main(step):
    #getTPSLfrom_telegram()
    try:
        x = threading.Thread(target=check_positions_new(symbol))
        x.start()
        y = threading.Thread(target=get_profit(symbol))
        y.start()   
    except :
        print('\n\nSomething went wrong. Continuing...')

In [ ]:
telegram_delay=12
bot_token='5454990445:AAGzfeY80oeI-1su0MEIRVuvTzBYIVlZIXU'
chat_id='92428939'

def getTPSLfrom_telegram():
    strr='https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requests.get(strr)
    rs=response.json()
    exit=0

    if(len(rs['result'])>0):
        rs2=rs['result'][-1]
        rs3=rs2['message']
        textt=rs3['text']
        datet=rs3['date']        
        if(time.time()-datet)<telegram_delay:
            if 'exit' in textt:
                exit=1
                telegram_bot_sendtext('Right now! Sir')
            if 'profit' in textt:
                trades = client.futures_account_trades(symbol=symbol)
                dataset=pd.DataFrame(trades).iloc[-50:]
                bnbdata=dataset[dataset['commissionAsset']=='BNB']
                busddata=dataset[dataset['commissionAsset']==asset]
                bnbcom=bnbdata['commission'].astype(float).sum()
                busdcom=busddata['commission'].astype(float).sum()
                com=(bnbcom*270+busdcom)*-1
                real=dataset['realizedPnl'].astype(float).sum()
                profit=com+real
                telegram_bot_sendtext('Profit = ' + str(profit))

    return(exit)

In [ ]:
def telegram_bot_sendtext(bot_message):
    bot_token2 = bot_token
    bot_chatID = chat_id
    send_text = 'https://api.telegram.org/bot' + bot_token2 + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

In [ ]:
def prt(message):
    # telegram message
    telegram_bot_sendtext(pointer+': '+message)
    print(pointer+': '+message)

In [ ]:
def start_stop():
    starttime=time.time()
    counterr=1
    timeout = time.time() + 60*60*6  # 60 seconds times 60 meaning the script will run for 12 hr
    while time.time() <= timeout:
        try:

            print("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
            main(counterr)
            counterr=counterr+1
            # stop=getTPSLfrom_telegram()
            # if stop==1:
            #     break
            if counterr>3:
                # break
                counterr=1
            time.sleep(4 - ((time.time() - starttime) % 4.0)) # 1 minute interval between each new execution
        except KeyboardInterrupt:
            print('\n\KeyboardInterrupt. Stopping.')
        

In [ ]:
start_stop()